In [4]:
import pandas as pd
import os

# 1. LOAD DATA
print("--- Loading Data ---")

# Load Master Country List (to ensure every country is included)
try:
    original_df = pd.read_csv("text_countries_edited.csv").fillna("")
    # Get unique sorted list of countries
    master_country_list = sorted([c for c in original_df['Country'].unique() if c != ""])
    print(f"Loaded master list: {len(master_country_list)} countries.")
except FileNotFoundError:
    print("Error: 'text_countries_edited.csv' not found.")
    master_country_list = []

# Load Entities Data
try:
    entities_df = pd.read_csv("entities_to_edit.csv").fillna("")
    print(f"Loaded entities: {len(entities_df)} rows.")
except FileNotFoundError:
    print("Error: 'entities_to_edit.csv' not found.")
    entities_df = pd.DataFrame()

# 2. RUN QH CATEGORY ANALYSIS
if not entities_df.empty and master_country_list:
    
    print("\n--- Starting QH Category Analysis (Top Level Only) ---")
    analysis_results = []

    for country in master_country_list:
        # Filter data for this specific country
        country_data = entities_df[entities_df['Country'] == country]
        
        if country_data.empty:
            # CASE: No entities found at all for this country
            analysis_results.append({
                "Country": country,
                "QH_Category": "No Data",
                "Frequency": 0
            })
        else:
            # CASE: Entities exist
            # Group strictly by 'qh_category'
            # dropna=False ensures we count rows even if the category is blank
            counts = country_data.groupby(['qh_category'], dropna=False).size().reset_index(name='Frequency')
            
            # Sort by Frequency (descending)
            counts = counts.sort_values(by='Frequency', ascending=False)
            
            for _, row in counts.iterrows():
                analysis_results.append({
                    "Country": country,
                    "QH_Category": row['qh_category'],
                    "Frequency": row['Frequency']
                })

    # 3. EXPORT
    output_filename = "country_qh_category_analysis.csv"
    final_df = pd.DataFrame(analysis_results)
    
    # Save to CSV
    final_df.to_csv(output_filename, index=False)
    
    print(f"Success! Analysis saved to '{output_filename}'")
    
    # Preview
    print("\nPreview of Analysis:")
    print(final_df.head(10))

else:
    print("\nAnalysis skipped. Please check your input files.")

--- Loading Data ---
Loaded master list: 4 countries.
Loaded entities: 739 rows.

--- Starting QH Category Analysis (Top Level Only) ---
Success! Analysis saved to 'country_qh_category_analysis.csv'

Preview of Analysis:
  Country                    QH_Category  Frequency
0     CAN                     Government         63
1     CAN                   not an actor         56
2     CAN                       Academia         16
3     CAN                  Civil Society          9
4     CAN                       Industry          6
5     CAN                  not specified          5
6     CAN  not relevant (from Footnotes)          2
7     CAN                   not specfied          2
8     GER                                       301
9     GER                  not specified          2
